<a href="https://colab.research.google.com/github/rahulsarkar906/KerasTuner_Hyperparamater/blob/main/Keras_tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install keras-tuner

     |████████████████████████████████| 96 kB 3.1 MB/s 
  Created wheel for kt-legacy: filename=kt_legacy-1.0.3-py3-none-any.whl size=9568 sha256=56dcf2710e972dd38f6fbb99e570f0f7aa0274d3eb08c0b55321dc3cd3c3e2b9
  Stored in directory: /root/.cache/pip/wheels/38/5c/e0/13003e68c17f403af40b92a24d20171b95fef13b0fdaba833c
Successfully built kt-legacy


In [ ]:
import math
import kerastuner as kt
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import Model
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.losses import MeanSquaredLogarithmicError

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  


In [ ]:
TRAIN_PATH = "/content/sample_data/california_housing_train.csv"
TEST_PATH = "/content/sample_data/california_housing_test.csv"

In [ ]:
target_column = 'median_house_value'

In [ ]:
# read the training and test data
train_data = pd.read_csv(TRAIN_PATH)

In [ ]:
test_data = pd.read_csv(TEST_PATH)

In [ ]:
# split the data into features and target
x_train, y_train = train_data.drop(target_column, axis=1), train_data[target_column]

In [ ]:
x_test, y_test = test_data.drop(target_column, axis=1), test_data[target_column]

In [ ]:
def scale_datasets(x_train, x_test):
  """
  Standard Scale test and train data
  Z - Score normalization
  """
  standard_scaler = StandardScaler()
  x_train_scaled = pd.DataFrame(
      standard_scaler.fit_transform(x_train),
      columns=x_train.columns
  )
  x_test_scaled = pd.DataFrame(
      standard_scaler.transform(x_test),
      columns = x_test.columns
  )
  return x_train_scaled, x_test_scaled

In [ ]:
# scale the dataset
x_train_scaled, x_test_scaled = scale_datasets(x_train, x_test)

In [ ]:
msle = MeanSquaredLogarithmicError()

In [ ]:
def build_model(hp):
  model = tf.keras.Sequential()
  # Tune the number of units in the dense layer
   # Choose an optimal value between 32-512
  hp_units1 = hp.Int('units1', min_value=32, max_value=512, step=32)
  hp_units2 = hp.Int('units2', min_value=32, max_value=512, step=32)
  hp_units3 = hp.Int('units3', min_value=32, max_value=512, step=32)
  model.add(Dense(units=hp_units1, activation='relu'))
  model.add(tf.keras.layers.Dense(units=hp_units2, activation='relu'))
  model.add(tf.keras.layers.Dense(units=hp_units3, activation='relu'))
  model.add(Dense(1, kernel_initializer='normal', activation='linear'))
  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(
      optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
      loss=msle,
      metrics=[msle]
  )

  return model

In [ ]:
# HyperBand algorithm from keras tuner
tuner = kt.Hyperband(
    build_model,
    objective='val_mean_squared_logarithmic_error',
    max_epochs=10,
    directory='keras_tuner_dir',
    project_name="keras_tuner_demo"
)

INFO:tensorflow:Reloading Oracle from existing project keras_tuner_dir/keras_tuner_demo/oracle.json


In [ ]:
tuner.search(x_train_scaled, y_train, epochs=10, validation_split=0.2)

Trial 30 Complete [00h 00m 41s]
val_mean_squared_logarithmic_error: 0.25117531418800354

Best val_mean_squared_logarithmic_error So Far: 0.10468193888664246
Total elapsed time: 00h 04m 07s
INFO:tensorflow:Oracle triggered exit


In [ ]:
for h_param in [f"units{i}" for i in range(1,4)] + ['learning_rate']:
  print(h_param, tuner.get_best_hyperparameters()[0].get(h_param))

units1 96
units2 64
units3 256
learning_rate 0.01


In [ ]:
best_model = tuner.get_best_models()[0]
best_model.build(x_train_scaled.shape)
best_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (17000, 96)               864       
_________________________________________________________________
dense_1 (Dense)              (17000, 64)               6208      
_________________________________________________________________
dense_2 (Dense)              (17000, 256)              16640     
_________________________________________________________________
dense_3 (Dense)              (17000, 1)                257       
Total params: 23,969
Trainable params: 23,969
Non-trainable params: 0
_________________________________________________________________


In [ ]:
best_model.fit(
    x_train_scaled, 
    y_train,
    epochs=10,
    batch_size=64
)

Epoch 1/10
266/266 [==============================] - 1s 2ms/step - loss: 0.0870 - mean_squared_logarithmic_error: 0.0870
Epoch 2/10
266/266 [==============================] - 1s 2ms/step - loss: 0.0854 - mean_squared_logarithmic_error: 0.0855
Epoch 3/10
266/266 [==============================] - 1s 2ms/step - loss: 0.0841 - mean_squared_logarithmic_error: 0.0841
Epoch 4/10
266/266 [==============================] - 1s 2ms/step - loss: 0.0837 - mean_squared_logarithmic_error: 0.0836
Epoch 5/10
266/266 [==============================] - 1s 2ms/step - loss: 0.0826 - mean_squared_logarithmic_error: 0.0826
Epoch 6/10
266/266 [==============================] - 1s 2ms/step - loss: 0.0820 - mean_squared_logarithmic_error: 0.0820
Epoch 7/10
266/266 [==============================] - 1s 2ms/step - loss: 0.0821 - mean_squared_logarithmic_error: 0.0821
Epoch 8/10
266/266 [==============================] - 1s 2ms/step - loss: 0.0812 - mean_squared_logarithmic_error: 0.0812
Epoch 9/10
266/266 [====

In [ ]:
# mean squared logarithmic error
msle(y_test, best_model.predict(x_test_scaled)).numpy()

0.54839236